In [38]:
import requests
import pandas as pd
import datetime as dt
import io

### Задача №1 

Выполнить сравнение объемов (в тоннах) региональных биржевых и региональных внебиржевых индексов Бензина Регуляр-92 по месяцам за весь период расчета (в виде таблицы)

** Замечание: ** * чтобы задача была более интересной, добавил сравнение объемов по center_code *

In [49]:
# URL значений региональных биржевых индексов за всю историю
CSV_ERI = 'http://spimex.com/indexes/service_functions.php?request=csv&index=eri'

# URL значений региональных внебиржевых индексов за всю историю
CSV_ORI = 'http://spimex.com/indexes/service_functions.php?request=csv&index=ori'

In [50]:
def upload_csv(csv_url):
    """Загружает, обрабатывает и возвращает csv-файл."""
    download=requests.get(csv_url).content
    csv=pd.read_csv(io.StringIO(download.decode('utf-8')), header=0, sep=';',
                   converters={"volume":int})
    return csv    
    
    

In [51]:
eri=upload_csv(CSV_ERI)
ori=upload_csv(CSV_ORI)

Убеждаемся в том, что файл прочитан верно. Выведем первые пять строк:

In [52]:
eri.head()

,date,index_code,product_code,center_code,index_value,index_value_per_liter,volume,value,num_trades,Unnamed: 9
0,2012-01-10,ERI_MOS_DTM,DTM,MOS,27946,"23,1952",6180,172706280,9,NaN
1,2012-01-10,ERI_MOS_DTZ,DTZ,MOS,32448,"26,7696",660,21415800,2,NaN
2,2012-01-10,ERI_MOS_REG,REG,MOS,26339,"19,6226",5700,150132900,17,NaN
3,2012-01-10,ERI_ROS_REG,REG,ROS,27128,"20,2104",4140,112309860,8,NaN
4,2012-01-10,ERI_SPB_DTM,DTM,SPB,28552,"23,6982",6180,176451360,9,NaN


Приводим данные к необходимому для решения задачи формату:

In [53]:
# Оставляем только Регуляр-92
eri = eri[eri.product_code == 'REG']

# Приводим столбец date в формат datetime
eri['date'] = pd.to_datetime(eri['date'], format='%Y-%m-%d')

#Проводим аналогичные манипуляции с ori
ori = ori[ori.product_code == 'REG']

ori['date'] = pd.to_datetime(ori['date'], format='%Y-%m-%d')

In [59]:
# Проводим слияние (outer join) eri и ori по date и center_code
merged = pd.merge(eri, ori, on=['date', 'center_code'], \
                  suffixes=('_eri', '_ori'), how = 'outer')

# Заполняем пустые значения volume нулями
merged[['volume_eri','volume_ori']] = merged[['volume_eri','volume_ori']].fillna(0)

# Дополняем таблицу дополнительными столбцами year и month для последующей
# группировки
merged['year'] = merged['date'].map(lambda x: x.year)
merged['month'] = merged['date'].map(lambda x: x.month)

In [75]:
result = \
    merged.groupby(by=['year', 'month', 'center_code'])\
    [['volume_eri', 'volume_ori']].\
    sum().reset_index()


In [76]:
result

,year,month,center_code,volume_eri,volume_ori
0,2012,1,EKA,2040.0,0.0
1,2012,1,IRK,2040.0,0.0
2,2012,1,MOS,39360.0,0.0
3,2012,1,ROS,22140.0,0.0
4,2012,1,SAM,3060.0,0.0
5,2012,1,SPB,18120.0,0.0
6,2012,2,EKA,60660.0,0.0
7,2012,2,IRK,60660.0,0.0
8,2012,2,MOS,65700.0,0.0
9,2012,2,NOS,51660.0,0.0


In [77]:
# Оформляем результат в виде xlsx-файла: 

writer = pd.ExcelWriter('reg.xlsx')
result.to_excel(writer, 'Sheet1', index=False)
writer.save()